## Developing segmentation related codes and classes


In [1]:
%load_ext autoreload
%autoreload 2
from transformers import AutoTokenizer, AutoModel, utils
#from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *

%load_ext autoreload
%autoreload 2

eskape_fasta = '/scratch/fastscratch/NBL/training_datasets/raw_sequence_sets/ncbi_repr/eskape_repr2023.fasta'

contigs = load_contigs([eskape_fasta],IsAddHeader=True,AsDataFrame=True, adding_reverse_complement=False)
fasta_ids = ['NC_000913.3', 'NC_007795.1', 'NC_002516.2']
sample3 = contigs[contigs.fasta_id.isin(fasta_ids)]



2023-08-10 14:00:25,041 - INFO - Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-08-10 14:00:25,042 - INFO - NumExpr defaulting to 8 threads.
2023-08-10 14:00:25,615 - INFO - Loading sequence data into memory!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
# Dumping fasta

random.seed(3)

dumping_path = '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/ESKAPE_sample.fasta'

Nlseq_min = 2300
Nlseq_max = 4300

seq_records = []  # List to hold SeqRecord objects
for i, rec in sample3.iterrows():

    act_seq = rec['sequence']
    act_fasta_id = rec['fasta_id']
    act_descr = rec['description']

    random_sequence_length = Nlseq_min + int((Nlseq_max- Nlseq_min)*random.random())
    random_start_pos = int((len(act_seq)-random_sequence_length)*random.random())
    random_end_pos = random_start_pos + random_sequence_length
    sample_seq = act_seq[random_start_pos:random_end_pos]
    sample_description = f'{act_descr} Sample start: {random_start_pos} end {random_end_pos}'

    new_id = f'{act_fasta_id}_{random_start_pos}_{random_end_pos}'

     # Create a Seq object from the sample_seq string
    seq = Seq(sample_seq)
    
    # Create a SeqRecord object with the Seq, id, and description
    seq_record = SeqRecord(seq, id=new_id, description=sample_description)
    
    # Append the SeqRecord object to the list
    seq_records.append(seq_record)
    
    print(new_id)
    
# Write the list of SeqRecord objects to a fasta file
with open(dumping_path, 'w') as output_handle:
    SeqIO.write(seq_records, output_handle, 'fasta')

print(f"Fasta file '{dumping_path}' created with {len(seq_records)} sequences.")
    
    
    

NC_000913.3_2524612_2527387
NC_007795.1_1702041_1705080
NC_002516.2_410266_413817
Fasta file '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/ESKAPE_sample.fasta' created with 3 sequences.


## Most itt csinálunk N darab fasta file-t a párhuzamos tokenizálás és pretraining létrehozására




In [6]:
N_files = 11
dumping_path = '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/pretraining/'

base_filename = 'ESKAPE_sample'
Nlseq_min = 2300
Nlseq_max = 4300
random.seed(3)

for i in range(N_files):
    act_filename = base_filename + str(i).zfill(2) + '.fasta'
    print(act_filename)
    output_filename = join(dumping_path, act_filename)
    seq_records = []  # List to hold SeqRecord objects
    for i, rec in sample3.iterrows():
        act_seq = rec['sequence']
        act_fasta_id = rec['fasta_id']
        act_descr = rec['description']
    
        random_sequence_length = Nlseq_min + int((Nlseq_max- Nlseq_min)*random.random())
        random_start_pos = int((len(act_seq)-random_sequence_length)*random.random())
        random_end_pos = random_start_pos + random_sequence_length
        sample_seq = act_seq[random_start_pos:random_end_pos]
        sample_description = f'{act_descr} Sample start: {random_start_pos} end {random_end_pos}'
    
        new_id = f'{act_fasta_id}_{random_start_pos}_{random_end_pos}'
    
         # Create a Seq object from the sample_seq string
        seq = Seq(sample_seq)
        
        # Create a SeqRecord object with the Seq, id, and description
        seq_record = SeqRecord(seq, id=new_id, description=sample_description)
        
        # Append the SeqRecord object to the list
        seq_records.append(seq_record)
        print(new_id)
    
    # Write the list of SeqRecord objects to a fasta file
    with open(output_filename, 'w') as output_handle:
        SeqIO.write(seq_records, output_handle, 'fasta')
    print(f"Fasta file '{output_filename}' created with {len(seq_records)} sequences.")




ESKAPE_sample00.fasta
NC_000913.3_2524612_2527387
NC_007795.1_1702041_1705080
NC_002516.2_410266_413817
Fasta file '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/pretraining/ESKAPE_sample00.fasta' created with 3 sequences.
ESKAPE_sample01.fasta
NC_000913.3_3885292_3887618
NC_007795.1_660471_663289
NC_002516.2_2943902_2948193
Fasta file '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/pretraining/ESKAPE_sample01.fasta' created with 3 sequences.
ESKAPE_sample02.fasta
NC_000913.3_2209173_2213145
NC_007795.1_424404_427982
NC_002516.2_5434688_5438257
Fasta file '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/pretraining/ESKAPE_sample02.fasta' created with 3 sequences.
ESKAPE_sample03.fasta
NC_000913.3_3438152_3441498
NC_007795.1_180422_184064
NC_002516.2_3700630_3704446
Fasta file '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/pretraining/ESKAPE_sample03.fasta' created with 3 sequences.
ESKAPE_sample04.fasta
NC_000913.3_143855_146757
NC_00779